<a href="https://colab.research.google.com/github/MelissaTESSA/Development-of-a-word-embedding-model-for-ARABIZI-using-fasttext/blob/main/Code_Entrainement_for_ARABIZI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
#we trained the model on several parameters : 
  input             # training file path (required)
    lr                # learning rate [0.1] 0.01-1
    dim               # size of word vectors [100] 100-300
    ws                # size of the context window [5]
    epoch             # number of epochs [5] 5-30
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0] 2
    maxn              # max length of char ngram [0] 5
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1] 2-3
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []
'''

"\n#we trained the model on several parameters : \n  input             # training file path (required)\n    lr                # learning rate [0.1] 0.01-1\n    dim               # size of word vectors [100] 100-300\n    ws                # size of the context window [5]\n    epoch             # number of epochs [5] 5-30\n    minCount          # minimal number of word occurences [1]\n    minCountLabel     # minimal number of label occurences [1]\n    minn              # min length of char ngram [0] 2\n    maxn              # max length of char ngram [0] 5\n    neg               # number of negatives sampled [5]\n    wordNgrams        # max length of word ngram [1] 2-3\n    loss              # loss function {ns, hs, softmax, ova} [softmax]\n    bucket            # number of buckets [2000000]\n    thread            # number of threads [number of cpus]\n    lrUpdateRate      # change the rate of updates for the learning rate [100]\n    t                 # sampling threshold [0.0001]\n    l

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install fasttext

In [ ]:
import pandas as pd
import fasttext

In [ ]:
#some similarity metrics, the one we used is the cosine similarity function, it was observed and proven that it gives good results on semantical similarities between wr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import cdist

def sklearn_cosine(x,y):
  x=x.reshape(1,-1)
  y=y.reshape(1,-1)
  return cosine_similarity(x, y)

def euclidian(x,y):
  x=x.reshape(1,-1)
  y=y.reshape(1,-1)
  return euclidean_distances(x, y)

def cosine(x,y):
  x=x.reshape(1,-1)
  y=y.reshape(1,-1)
  return cosine_distances(x, y)

def scipy_cosine():
    return 1. - cdist(x, y, 'cosine')

In [ ]:
#cosine similarity result
def sim_cos(dico,n):
  avg = 0
  if n==0:
    for k in dico.keys():
      x=model.get_word_vector(k)
      y=model.get_word_vector(dico[k])
      result = sklearn_cosine(x,y)[0,0]
      if result<0.61:
        avg=avg+1
  else:
    for k in dico.keys():
      x=model.get_word_vector(k)
      y=model.get_word_vector(dico[k])
      result = sklearn_cosine(x,y)[0,0]
      if result>=0.61:
        avg=avg+1
  avg = avg/len(dico)
  print("***************")
  print("Precision of the model on",len(dico),"examples using cosine_similarity metric :",avg)

#cosine distance
def cos_dis(dico):
  avg = 0
  for k in dico.keys():
    x=model.get_word_vector(k)
    y=model.get_word_vector(dico[k])
    result = cosine(x,y)[0,0]
    avg=avg+result
    print(k,"---",dico[k],":",result)
  avg = avg/len(dico)
  print("*********\nAverge cosine dis result of",len(dico),"examples :",avg)

#euclidian distance
def sim_euc(dico):
  avg = 0
  for k in dico.keys():
    x=model.get_word_vector(k)
    y=model.get_word_vector(dico[k])
    result = euclidian(x,y)[0,0]
    avg=avg+result
    #print(k,"---",dico[k],":",result)
  avg = avg/len(dico)
  print("*********\nAverge euclidian result of",len(dico),"examples :",avg)

#jaccard score
def jacc_score(dico):
  avg = 0
  for k in dico.keys():
    x=model.get_word_vector(k)
    y=model.get_word_vector(dico[k])
    result = jaccard_score(x, y, average='samples')
    avg=avg+result
    print(k,"---",dico[k],":",result)
  avg = avg/len(dico)
  print("*********\nAverge jaccard score of",len(dico),"examples :",avg)

In [ ]:
#similarity test function, takes in input a dictionary X and a label binary vector Y (Y is 0 when the corresponding couple isn't similar, 1 if there's similarity)
def similarity(X,Y, threshold):
  avg = 0
  i = 0
  for k in X.keys():
    x=model.get_word_vector(k)
    y=model.get_word_vector(X[k])
    result = sklearn_cosine(x,y)[0,0]
    if result<threshold:
      v=0
    else:
      v=1
    if v == Y[i]:
      avg+=1
    #print(k,"---",X[k],":",result,"\t\t\t\t",v,"--",Y[i])
    i+=1
  print("Number of true positives :",avg, "/",len(X),"examples")
  avg = avg/len(X)
  print("Precision of the model on",len(X),"examples using cosine function :",avg)
  #return avg

In [ ]:
#model +
#'skipgram',minn=1, maxn=4, wordNgrams=1, dim=30, epoch =2, lr = 0.18500000000000008 --> precision on the test set : 0.7613636363636364
model = fasttext.train_unsupervised("/content/drive/Shareddrives/Arabizi project /dataset/Training_Dataset_ARABIZI.csv", 'skipgram',minn=1, maxn=4, wordNgrams=1, dim=30, epoch =2, lr = 0.18500000000000008)
model.save_model("/content/drive/Shareddrives/Arabizi project /dataset/final/model.bin")

In [ ]:
similarity(X,Y,0.61) #returns the precision on the test set 88 examples using cosine_similarity metric
#sim_cos(dico,0)      #returns the precision on the test set 44 not similar examples using cosine_similarity metric
#sim_cos(dico2,1)     #returns the precision on the test set 44 similar examples using cosine_similarity metric

In [ ]:
#Test set 2
X = {"dar":"maison",
      "samet":"bassel",
      "wled":"tfel",
      "ghaba":"chjer",
      "mra":"femme",
      "rajel":"homme",
      "ayetli":"kelemni",
      "bnadem":"eabd",
      "cahier":"ktab",
      "tomobil":"karoussa",
      "kebch":"aid",
      "bligha":"sebat",
      "zroudia":"batata",
      "bhar":"yaoum",
      "hayawan":"kebch",
      "korsi":"yekeoud",
      "hlib":"bakara",
      "yema":"mama",
      "nrouh":"nemchi",
      "yheb":"yebghi",
      "ykoul":"yehder",
      "mliha":"hayla",
      "yekra":"yekteb",
      "ysegem":"yaedel",
      "lyoum":"aujourd'hui",
      "tbib":"mrid",
      "marche":"tekdi",
      "boucher":"lham",
      "lham":"begri",
      "regarde":"chouf",
      "azul":"salam",
      "tefla":"chira",
      "saha":"choukran",
      "aaref":"alem",
      "kbir":"adjouz",
      "chir":"tfel",
      "yechti":"yebghi",
      "yedjyah":"yeqbah",
      "mrid":"sbitar",
      "ydawi":"dwa",
      "hadja":"aefsa",
      "yghres":"chjer",
      "kelb":"chien",
      "nsksik":"question",
     #
      "bab":"nrouh",
      "hchich":"hmar",
      "chargeur":"dar",
      "stylo":"rajel",
      "zdjadj":"hchich",
      "gaz":"nhel",
      "farmadj":"cartable",
      "chachra":"internet",
      "banana":"ktab",
      "ghali":"beid",
      "saedni":"houma",
      "ktab":"tefla",
      "chejra":"mra",
      "poto":"babour",
      "hout":"zbel",
      "taka":"bligha",
      "chaba":"fletox",
      "hram":"bacina",
      "poupoun":"valiza",
      "hadjra":"yaourt",
      "carlaja":"scotch",
      "rkhis":"mani",
      "sma":"chargeur",
      "hchicha":"mra",
      "makla":"couronne",
      "srir":"khedma",
      "rassek":"newar",
      "sport":"baba",
      "chitan":"haraqa",
      "tfel":"prise",
      "taam":"bidoun",
      "zbel":"drap",
      "zaaim":"kavi",
      "mous":"veste",
      "ktabat":"saboun",
      "radjel":"intelligent",
      "triko":"ghaba",
      "rqad":"maaen",
      "fenyan":"telephone",
      "djay":"khimar",
      "conne":"tefla",
      "bassel":"maaen",
      "khfaf":"khimar",
      "meftah":"calculatrice",
      }

Y=[]
for i in range(44):
  Y.append(1)
for i in range(45):
  Y.append(0)
#
print(len(Y))
print(len(X))
#print(len(X_n))

In [ ]:
#replaces the autotune-validation feature, it loops over all possible parameters (well defined range), calculate the precision for each model, and returns the best one with the 
#corresponding parameters
def autotune():
  #takes a training set in input
  #train it and variate the parameters in a defined range for each
  #returns at each training the score on a test set of 90 labaled examples
  #we take the best score
  best = 0
  cpt = 0 #compteur de la boucle
  for epoch in range(2,30,2):
      for dim in range(5,50,5):
        for wordngrams in range(1,6):
          for minn in range(1,4):
            for maxn in range(1,minn+4):
              lr=0
              for i in range(10):
                cpt +=1 
                print("--",cpt,"--")
                lr = lr + 0.02
                model = fasttext.train_unsupervised("/content/drive/Shareddrives/Arabizi project /dataset/Training_Dataset_ARABIZI.csv", 'skipgram',minn=minn, maxn=maxn, wordNgrams=wordngrams, dim=dim, epoch = epoch, lr = lr)
                res = similarity(X,Y,0.6)
                if res>best:
                  best=res
                  e=epoch
                  d=dim
                  wn=wordngrams
                  mn=minn
                  mx=maxn
                  l=lr
                  print("*********\nAverge cosine result of",len(X),"examples :",res)
                  print("epoch:",epoch,'dim:',dim,"wngram:",wordngrams,"minn:",minn,"maxn:",maxn,"lr:",lr)
  print("\nbest score is :",best,"at epoch:",e,'dim:',d,"wngram:",wn,"minn:",mn,"maxn:",mx,"lr:",l)

#execute
autotune()

In [ ]:
#reduce dimensionality and plot results
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(dict):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for key,value in dict.items():
        tokens.append(model.get_word_vector(key))
        labels.append(key)

        tokens.append(model.get_word_vector(value))
        labels.append(value)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=250, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
        plt.xlim(-10,10)
    plt.show()


In [ ]:
#plot test
t_dico ={"dar":"maison",
      "samet":"bassel",
      "wled":"tfel",
      "ghaba":"chjer",
      "mra":"femme",
      "rajel":"homme",
      "ayetli":"kelemni",
      "bnadem":"eabd",
      "cahier":"ktab",
      "tomobil":"karoussa",
      "kebch":"aid",
      "bligha":"sebat",}
tsne_plot(t_dico)

In [ ]:
#Test Set
#How to proceed : list of words that we seek for their nearest neighbors with our model
dico ={"bab":"nrouh",
      "hchich":"hmar",
      "chargeur":"dar",
      "stylo":"rajel",
      "zdjadj":"hchich",
      "gaz":"nhel",
      "farmadj":"cartable",
      "chachra":"internet",
      "banana":"ktab",
      "ghali":"beid",
      "saedni":"houma",
      "ktab":"tefla",
      "chejra":"mra",
      "poto":"babour",
      "hout":"zbel",
      "taka":"bligha",
      "chaba":"fletox",
      "hram":"bacina",
      "poupoun":"valiza",
      "hadjra":"yaourt",
      "carlaja":"scotch",
      "rkhis":"mani",
      "sma":"chargeur",
      "hchicha":"mra",
      "makla":"couronne",
      "srir":"khedma",
      "rassek":"newar",
      "sport":"baba",
      "chitan":"haraqa",
      "tfel":"prise",
      "taam":"bidoun",
      "zbel":"drap",
      "zaaim":"kavi",
      "mous":"veste",
      "ktabat":"saboun",
      "radjel":"intelligent",
      "triko":"ghaba",
      "rqad":"maaen",
      "fenyan":"telephone",
      "djay":"khimar",
      "conne":"tefla",
      "bassel":"maaen",
      "khfaf":"khimar",
      "meftah":"calculatrice",
      }
_dico ={"dar":"maison",
      "samet":"bassel",
       "auto":"karoussa",
       "yheb":"yebghi",
      "ykoul":"yehder",
      "mliha":"hayla",
      "yekra":"yekteb",
      "ydir":"ywasi",
      "lyoum":"aujourd'hui",
      }
dico2 ={"dar":"maison",
      "samet":"bassel",
      "wled":"tfel",
      "ghaba":"chjer",
      "mra":"femme",
      "rajel":"homme",
      "ayetli":"kelemni",
      "bnadem":"eabd",
      "cahier":"ktab",
      "tomobil":"karoussa",
      "kebch":"aid",
      "bligha":"sebat",
      "zroudia":"batata",
      "bhar":"yaoum",
      "hayawan":"kebch",
      "korsi":"yekeoud",
      "hlib":"bakara",
      "yema":"mama",
      "nrouh":"nemchi",
      "yheb":"yebghi",
      "ykoul":"yehder",
      "mliha":"hayla",
      "yekra":"yekteb",
      "ysegem":"yaedel",
      "lyoum":"aujourd'hui",
      "tbib":"mrid",
      "marche":"tekdi",
      "boucher":"lham",
      "lham":"begri",
      "regarde":"chouf",
      "azul":"salam",
      "tefla":"chira",
      "saha":"choukran",
      "aaref":"alem",
      "kbir":"adjouz",
      "chir":"tfel",
      "yechti":"yebghi",
      "yedjyah":"yeqbah",
      "mrid":"sbitar",
      "ydawi":"dwa",
      "hadja":"aefsa",
      "yghres":"chjer",
      "kelb":"chien",
      "nsksik":"question",
      }

In [ ]:
try:
  print(model.words)
  print(len(model.words))
except:
  print('exception')

**************************************************************************
down here are some util functions
**************************************************************************

In [ ]:
#cosine smilarity function
#import modules
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine(x,y):
  return dot(x, y)/(norm(x)*norm(y))
#define arrays
x = np.array([1, 1, 1, 1, 0, 0, 0, 0, 0]) 
y = np.array([0, 0, 1, 1, 1, 1, 0, 0, 0])

#calculate Cosine Similarity python
result = cosine(x,y)

print("The Cosine Similarity between two vectors is: ",result)

#sklearn.metrics.pairwise.cosine_similarity(X, Y=None, dense_output=True)

In [ ]:
#print resulting vectors into a matrix
import numpy as np
words = model.words
t = []
for w in words[:5000]:
  v = model.get_word_vector(w)
  t.append(v)
M = np.array(t)
print(M.shape)
print(M)

In [ ]:
#split dataset into train, test, validation set
import pandas as pd
from sklearn.model_selection import train_test_split

train_ratio = 0.7, validation_ratio = 0.15, test_ratio = 0.15
dataX = pd.read_csv("/content/drive/Shareddrives/Arabizi project /TrainingFastText/Dataset_ARABIZI.csv", engine="python")
x_train, x_test = sk.model_selection.train_test_split(dataX, test_size=1 - train_ratio)
x_val, x_test = sk.model_selection.train_test_split(x_test, test_size=test_ratio/(test_ratio + validation_ratio)) 
print(len(x_train), len(x_val), len(x_test))
x_train.to_csv('train.csv')
x_val.to_csv('valid.csv')
x_test.to_csv('test.csv')